In [1]:
import os
from pyngrok import ngrok

# 設定 ngrok 金鑰並啟動隧道


# 設定 LINE 機器人金鑰和密鑰
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

import sqlite3

# 建立 Flask 應用
app = Flask(__name__)

# 配置 LINE 機器人
configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)

# 初始化 SQLite 資料庫連線
conn = sqlite3.connect('reports.db', check_same_thread=False)
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS reports (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        content TEXT NOT NULL
    )
''')
conn.commit()

@app.route("/", methods=['POST'])
def callback():
    # 獲取 X-Line-Signature 標頭的值
    signature = request.headers['X-Line-Signature']

    # 獲取請求主體作為文本
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # 處理 webhook 主體
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    # 解析用戶輸入的資料格式
    user_input = event.message.text.split('、')
    if len(user_input) == 2:
        title, content = user_input

        # 將資料儲存進資料庫
        cursor.execute('INSERT INTO reports (title, content) VALUES (?, ?)', (title, content))
        conn.commit()

        # 回復用戶
        reply_text = f"感謝您的回報 以下是您的回報標題: {title}"
    else:
        reply_text = "請按照格式輸入: 標題、內文"

    # 回復用戶訊息
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    app.run(port=port)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jun/2024 04:17:05] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U05f8fb23f03a1256e0ff3153571c9e24","events":[]}
